In [ ]:
# | default_exp blocks/se

# Imports

In [ ]:
# | export


import torch
from einops import rearrange
from torch import nn

from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.activations import get_act_layer
from vision_architectures.utils.custom_base_model import CustomBaseModel
from vision_architectures.utils.normalizations import get_norm_layer

# Config

In [ ]:
# | export


class SEBlock3DConfig(CustomBaseModel):
    dim: int
    r: float
    normalization: str = "batchnorm3d"
    activation: str = "silu"

# Architecture

In [ ]:
# | export


class SEBlock3D(nn.Module):
    def __init__(self, config: SEBlock3DConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = SEBlock3DConfig.model_validate(config | kwargs)

        dim = self.config.dim
        r = self.config.r
        activation = self.config.activation
        normalization = self.config.normalization

        excitation_dim = int(dim // r)

        self.squeeze = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.excitation = nn.Sequential(
            nn.Conv3d(
                dim, excitation_dim, kernel_size=1, bias=False if normalization.startswith("batchnorm") else True
            ),
            get_norm_layer(normalization, excitation_dim),
            get_act_layer(activation),
            nn.Conv3d(
                excitation_dim, dim, kernel_size=1, bias=False if normalization.startswith("batchnorm") else True
            ),
            get_norm_layer(normalization, dim),
            nn.Sigmoid(),
        )

        self.checkpointing_level2 = ActivationCheckpointing(2, checkpointing_level)

    def _forward(self, x: torch.Tensor, channels_first: bool = True):
        # x: (b, [dim], z, y, x, [dim])

        if not channels_first:
            x = rearrange(x, "b z y x d -> b d z y x").contiguous()
        # Now x is (b, dim, z, y, x)

        p = self.squeeze(x)
        # (b, dim, 1, 1, 1)
        p = self.excitation(p)
        # (b, dim, 1, 1, 1)
        x = x * p
        # (b, dim, z, y, x)

        if not channels_first:
            x = rearrange(x, "b d z y x d -> b z y x d").contiguous()
            # (b, z, y, x, dim)

        return x

    def forward(self, *args, **kwargs):
        return self.checkpointing_level2(self._forward, *args, **kwargs)

In [ ]:
test = SEBlock3D(dim=12, r=3)
display(test)

sample_input = torch.randn(2, 12, 4, 4, 4)
test(sample_input).shape


SEBlock3D(
  (squeeze): AdaptiveAvgPool3d(output_size=(1, 1, 1))
  (excitation): Sequential(
    (0): Conv3d(12, 4, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
    (1): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU()
    (3): Conv3d(4, 12, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
    (4): BatchNorm3d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Sigmoid()
  )
)

torch.Size([2, 12, 4, 4, 4])

# nbdev

In [ ]:
!nbdev_export